In [37]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import shutil
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Capturar video
cap = cv2.VideoCapture('vtest.avi')

# Parámetros para el flujo óptico
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Inicializar el modelo MOG2 para la sustracción de fondo
fgbg = cv2.createBackgroundSubtractorMOG2()

# Leer el primer frame
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Detección de características en el primer frame
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Crear una máscara para dibujar los puntos
mask = np.zeros_like(old_frame)

frame_count = 0
folder_name = "FOD_vtest"
if os.path.exists(folder_name):
    shutil.rmtree(folder_name)  # Eliminar la carpeta y su contenido
os.makedirs(folder_name)

freeman_data = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Aplicar sustracción de fondo usando MOG2
    fgmask = fgbg.apply(frame)

    # Aplicar umbral para la segmentación
    threshold = 50  # Ajusta este valor según tus necesidades
    _, thresh = cv2.threshold(fgmask, threshold, 255, cv2.THRESH_BINARY)

    # Encontrar contornos en la máscara segmentada
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtrar contornos por área
    min_contour_area = 100  # Ajusta este valor según tus necesidades
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    # Crear una nueva máscara para contornos filtrados
    mask_filtered_contours = np.zeros_like(frame)

    # Dibujar contornos filtrados en la nueva máscara
    cv2.drawContours(mask_filtered_contours, filtered_contours, -1, (255, 255, 255), thickness=cv2.FILLED)

    frame_count += 1

    if frame_count % 5 == 0:
        name = os.path.join(folder_name, f'frame{frame_count}.jpg')
        cv2.imwrite(name, mask_filtered_contours)

    if len(mask_filtered_contours.shape) > 2:
        mask_filtered_contours = cv2.cvtColor(mask_filtered_contours, cv2.COLOR_BGR2GRAY)

    # Cambiar el tipo de datos a CV_8UC1
    mask_filtered_contours = cv2.convertScaleAbs(mask_filtered_contours)

    # Aplicar umbral adaptativo a la máscara filtrada
    adaptive_thresh = cv2.adaptiveThreshold(mask_filtered_contours, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Guardar la imagen del umbral adaptativo
    if frame_count % 5 == 0:
        adaptive_thresh_filename = os.path.join(folder_name, f'adaptive_thresh_{frame_count}.jpg')
        cv2.imwrite(adaptive_thresh_filename, adaptive_thresh)

    # Encontrar contornos en la máscara
    contours, _ = cv2.findContours(adaptive_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Calcular el área promedio de los contornos
    average_contour_area = sum(cv2.contourArea(cnt) for cnt in contours) / len(contours)

    # Obtener la cadena de Freeman
    freeman_chain = []
    if len(contours) > 0:
        # Aproximar el contorno usando Douglas-Peucker
        epsilon = 0.02 * cv2.arcLength(contours[0], True)
        approx_contour = cv2.approxPolyDP(contours[0], epsilon, True)

        for i in range(1, len(approx_contour)):
            x_diff = approx_contour[i][0][0] - approx_contour[i - 1][0][0]
            y_diff = approx_contour[i][0][1] - approx_contour[i - 1][0][1]

            # Asignar dirección de Freeman (0 a 7)
            freeman_direction = (y_diff + 1) * 3 + (x_diff + 1)
            freeman_chain.append(freeman_direction)

        # Dibujar el contorno aproximado en la imagen de umbralización
        cv2.drawContours(adaptive_thresh, [approx_contour], 0, (255, 0, 0), 2)

        # Dibujar la cadena de Freeman en la imagen de umbralización
        font = cv2.FONT_HERSHEY_SIMPLEX
        for i, direction in enumerate(freeman_chain):
            cv2.putText(adaptive_thresh, str(direction), (approx_contour[i][0][0], approx_contour[i][0][1]),
                        font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        if frame_count % 5 == 0:
            freeman_filename = os.path.join(folder_name, f'freeman_{frame_count}.jpg')
            cv2.imwrite(freeman_filename, adaptive_thresh)

    # Almacena los puntos de la cadena de Freeman
    freeman_data.append(freeman_chain)

    
    # Dibujar solo los contornos filtrados
    frame_with_filtered_contours = frame.copy()
    cv2.drawContours(frame_with_filtered_contours, filtered_contours, -1, (0, 255, 0), 2)

    # Calcular flujo óptico
    p1, _, _ = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Seleccionar puntos válidos
    p1_valid = p1[~np.isnan(p1).any(axis=2)]
    good_new = p1_valid.reshape(-1, 1, 2)

    # Ensure p0 has the same shape as good_new
    p0 = p0[:good_new.shape[0], :, :]

    # Seleccionar puntos antiguos correspondientes a los puntos válidos
    good_old = p0[~np.isnan(p1).any(axis=2)]

    # Dibujar líneas y puntos en el frame
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 255, 0), -1)

    # Calcular el flujo óptico entre los puntos anteriores y nuevos
    flow = p1 - p0

    # Calcular la magnitud del flujo óptico
    magnitude = np.sqrt(np.sum(flow**2, axis=2))

    # Aplicar umbral para la segmentación
    threshold_optical_flow = 1.0  # Ajusta este valor según tus necesidades
    mask_segmentation_optical_flow = (magnitude > threshold_optical_flow).astype(np.uint8) * 255

    mask_segmentation_resized = cv2.resize(mask_segmentation_optical_flow, (frame.shape[1], frame.shape[0]))

    # Mostrar las imágenes (opcional, puedes comentar estas líneas si no las necesitas)
    cv2.imshow('Original Frame', frame)
    cv2.imshow('Moving Objects with Contours', frame_with_filtered_contours)
    cv2.imshow('Mask with Filtered Contours', mask_filtered_contours)
    cv2.imshow('Adaptive Threshold', adaptive_thresh)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Actualizar el marco anterior
    old_gray = frame_gray.copy()

# Liberar recursos
cv2.destroyAllWindows()
cap.release()

# Aplicar k-Means a los datos de la cadena de Freeman
freeman_data = np.array(freeman_data)
freeman_data_flat = [item for sublist in freeman_data for item in sublist]  # Aplanar la lista de listas
freeman_data_flat = np.array(freeman_data_flat).reshape(-1, 1)
freeman_data_scaled = StandardScaler().fit_transform(freeman_data_flat)

k = 3  # Número de clusters deseado
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(freeman_data_scaled)

colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0)]  # Rojo, verde, azul
cluster_colors = [colors[label] for label in clusters]

# Supongamos que 'adaptive_thresh' es tu imagen de umbralización
result_image = cv2.cvtColor(adaptive_thresh, cv2.COLOR_GRAY2BGR)

for i, contour in enumerate(filtered_contours):
    for point in contour:
        x, y = point[0]
        color = cluster_colors[i]
        cv2.circle(result_image, (int(x), int(y)), 3, color, -1)

        cv2.circle(result_image, (x, y), 3, color, -1)

for i, (new, old) in enumerate(zip(good_new, good_old)):
    a, b = new.ravel()
    c, d = old.ravel()
    color = cluster_colors[i]
    cv2.circle(result_image, (int(a), int(b)), 3, color, -1)



cv2.imshow('Clustered Image', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()





C:\Users\chj06\AppData\Local\Temp\ipykernel_4416\2232351365.py:177: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  freeman_data = np.array(freeman_data)
c:\Users\chj06\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# Visualizar imágenes usando Matplotlib
image_files = sorted([os.path.join(folder_name, f) for f in os.listdir(folder_name) if f.endswith('.jpg')])

for image_file in image_files:
    img = cv2.imread(image_file)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()